# Model Emphasis and Analysis Demo

This notebook demonstrates the key features of the model emphasis, analysis, and generation capabilities.

In [1]:
import mlx.core as mx
import mlx.nn as nn

from mi_experiments.utils.loading import load

import re
from dataclasses import dataclass
from typing import Dict, Optional, Union

from mlx_lm.tokenizer_utils import TokenizerWrapper


/Users/jluker/mi-experiments/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load the model
model, tokenizer = load("mlx-community/Mistral-7B-Instruct-v0.3-4bit")

# Utility functions
def create_additive_causal_mask(N: int, offset: int = 0):
    rinds = mx.arange(offset + N)
    linds = mx.arange(offset, offset + N) if offset else rinds
    mask = linds[:, None] < rinds[None]
    return mask * -1e9

@dataclass
class ModelArgs:
    model_type: str
    hidden_size: int
    num_hidden_layers: int
    intermediate_size: int
    num_attention_heads: int
    rms_norm_eps: float
    vocab_size: int
    head_dim: Optional[int] = None
    max_position_embeddings: Optional[int] = None
    num_key_value_heads: Optional[int] = None
    attention_bias: bool = False
    mlp_bias: bool = False
    rope_theta: float = 10000
    rope_traditional: bool = False
    rope_scaling: Optional[Dict[str, Union[float, str]]] = None
    tie_word_embeddings: bool = True

    @classmethod
    def from_model(cls, model: nn.Module):
        return cls(
            model_type=model.model_type,
            hidden_size=model.args.hidden_size,
            num_hidden_layers=model.args.num_hidden_layers,
            intermediate_size=model.args.intermediate_size,
            num_attention_heads=model.args.num_attention_heads,
            rms_norm_eps=model.args.rms_norm_eps,
            vocab_size=model.args.vocab_size,
            head_dim=model.head_dim,
            num_key_value_heads=model.n_kv_heads,
            tie_word_embeddings=model.args.tie_word_embeddings
        )

# Extract model arguments
model_args = ModelArgs.from_model(model)

# Ensure tokenizer is wrapped
if not isinstance(tokenizer, TokenizerWrapper):
    tokenizer = TokenizerWrapper(tokenizer)

def format_prompt(prompt: str) -> str:
    return f"[INST] {prompt} [/INST]"

def extract_generated_text(full_text: str) -> str:
    match = re.search(r'\[/INST\]\s*(.*)', full_text, re.DOTALL)
    return match.group(1).strip() if match else ""

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 125470.63it/s]


In [3]:
import string
import random
from mi_experiments.inference.batch_manager import batch_generate

capital_letters = string.ascii_uppercase
distinct_pairs = [(a, b) for i, a in enumerate(capital_letters) for b in capital_letters[i + 1:]]

num_prompts = 10
prompt_template = "Think of a real word containing both the letters {l1} and {l2}. Then, say 3 sentences which use the word."
prompts_raw = [prompt_template.format(l1=p[0], l2=p[1]) for p in random.sample(distinct_pairs, num_prompts)]
prompt_template_2 = "Come up with a real English word containing both the letters {l1} and {l2}. No acronyms. Then, give 3 complete sentences which use the word."
prompts_raw_2 = [prompt_template_2.format(l1=p[0], l2=p[1]) for p in random.sample(distinct_pairs, num_prompts)]

response = batch_generate(model, tokenizer, prompts=prompts_raw[:10]+prompts_raw_2[:10], max_tokens=20, verbose=True, temp=0.0)
# print(response)

Prompt: 270.917 tokens-per-second
Generation: 6710.119 tokens-per-second
Prompt: Think of a real word containing both the letters A and Y. Then, say 3 sentences which use the word.
Sure, here are three sentences using the word "any":

1. "Can you bring
Prompt: Think of a real word containing both the letters D and J. Then, say 3 sentences which use the word.
1. The dog jumped over the ditch.
2. The DJ played a jaz
Prompt: Think of a real word containing both the letters S and X. Then, say 3 sentences which use the word.
Sure, here are three sentences using the word "exotic":

1. "The exotic
Prompt: Think of a real word containing both the letters D and G. Then, say 3 sentences which use the word.
1. The dog is digging a hole in the garden.

2. The doctor gave
Prompt: Think of a real word containing both the letters C and T. Then, say 3 sentences which use the word.
1. The cat chased the mouse across the mat.
2. The teacher corrected the
Prompt: Think of a real word containing both the 